# Food Data Analytic (Cleaning)

#### Import Library

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

import re

%matplotlib inline

#### Functions

In [2]:
def get_null_rows(data):
    '''
    data: DataFrame
    this function is to get the row where
    '''
    drop_index = []
    for i, r in data.iterrows():
        if (data.loc[i].isnull().sum() == data.shape[1]
        or data.loc[i].isnull().sum() >= int(data.shape[1]*0.70)):
            drop_index.append(i)
    if len(drop_index) == 1:
        drop_index = drop_index[0]
        
    return drop_index

def get_section_title(data):
    '''
    data: Dataframe
    This function is to get the title of the table.
    It will return the index of the row where there is non null value
    '''
    null_df = data.loc[get_null_rows(data)]
    non_null_ind = []
    for index , row in null_df.iterrows():
        if null_df.loc[index].isnull().sum() != null_df.shape[1]:
            non_null_ind.append(index)
    if len(non_null_ind) == 1:
        non_null_ind = non_null_ind[0]
    return non_null_ind

In [3]:
def get_null_col(data, columns):
    
    '''
    data = dataframe
    columns = dataframe columns
    This function is to get the columns where each row has 70% null values
    '''
    drop_columns = []

    print("List of Columns to be dropped as they have 70% of null values")
    print()
    for col in columns:
        if data[col].isnull().sum() > (0.70*data.shape[0]):
            print(col, "   ",data[col].isnull().sum() )
            drop_columns.append(col)
        else:
            continue
    print("\nNumber of columns: ",len(drop_columns), "\n")
    if len(drop_columns) == 1:
        drop_columns = drop_columns[0]
    return drop_columns

In [4]:
def get_index_column(data, columns):
    '''
    data: DataFrame
    columns: columns to be replaced by
    This function is to get the row where it contains the columns and 
    '''
    columns_row_index = []
    for index, row in data.iterrows():
        for val in row:
            if val in columns:
                columns_row_index.append(index)
                break
    if len(columns_row_index) == 1:
        columns_row_index = columns_row_index[0]
    return columns_row_index

In [5]:
def get_columns(data, columns):
    '''
    data: DataFrame
    columns: Columns/Existing columns
    This function is to get the row where it contains columns
    '''
    new_columns = []
    for index, row in data.iterrows():
        if set(list(row)).intersection(columns):
            for item in list(row):
                new_columns.append(item)
    return new_columns

In [6]:
def renaming_columns(data, columns, new_columns):
    '''
    data: DataFrame
    columns: DataFrame Columns to be replaced
    new_columns: New columns name to be replace with
    
    This function is to renaming the columns
    '''
    column_mapper = {}
    for i in list(range(0,data.shape[1])):
        column_mapper[columns[i]] = new_columns[i]
    return column_mapper

def renaming_columns_map(data, dataColumns ,columns):
    '''
    data: DataFrame
    dataColumns: current DataFrame columns
    columns: Columns/Existing columns
    This function is to get the column mapper to rename columns
    '''
    new_columns = get_columns(data, columns)
    renaming_columns_map = {}
    for i in list(range(len(new_columns))):
        key = dataColumns[i]
        value = new_columns[i]
        renaming_columns_map[key]=value
    return renaming_columns_map

In [7]:
def get_date(x):
    try:
        pattern1 = re.compile("\d{4}-\d{2}-\d{2}")
        pattern2 = re.compile("\d+\/\d{2}\/\d{4}")
        if pattern1.match(x):
            result = re.search("\d{4}-\d{2}-\d{2}", x)
            result = result.group()
        elif pattern2.match(x):
            result = re.search("\d+\/\d{2}\/\d{4}", x)
            result = result.group()
        else:
            result = "none"
        return result
    except Exception as e:
        return "none"

def convert_to_datetime(date_col, format = '%d/%m/%Y'):
    converted_date_col = pd.to_datetime(date_col, format = '%d/%m/%Y', errors = 'coerce')
    return converted_date_col


def convert_to_datetime_v2(x):
    x = get_date(x)
    try:
        result = pd.to_datetime(x, format='%Y-%m-%d', errors = 'coerce')
        return result
    except Exception as e:
        result_2 = pd.to_datetime(x, format="%d/%m/%Y", errors = 'coerce')
        return result_2

In [8]:
def fill_na_val_for_date(date_col, pattern):
    '''
    date_col: Date Column
    Pattern: Specify Pattern
    Replacing the null values for the date column by the previous result
    '''
    date_list = list(date_col.astype(str))
    pattern = re.compile("\d{4}\-\d{2}\-\d{2}")
    new_date_list = []
    for ind, item in enumerate(date_list):
        if pattern.match(item):
            new_date_list.append(item)
        else:
            if ind == 0:
                item = date_list[ind+1]
                new_date_list.append(item)
            else:
                item = date_list[ind-1]
                new_date_list.append(item)
    return new_date_list

In [10]:
def get_bacteria(text):
    bacteria_lst = ["Escherichia coli", 
                    "Listeria", 
                    "Salmonella", 
                    "Campylobacter", 
                    "Clostridium", 
                    "Staphylococcus", 
                    "Shigellosis", 
                    "Vibrio vulnificus", 
                    "Histamine",
                   "Ciguatera",
                   "Iodine",
                   "Botulism",
                   "Enteric",
                   "Bacillus",
                   "Alphatoxin",
                   "Erythrosine",
                   "Propargite",
                   "Fipronil",
                    "Toxoplasma gondii",
                    "Giardia lamblia",
                    "Cryptosporidium",
                    "Ascaris lumbricoides",
                    "flukes",
                    "pinworms",
                    "Enterobiasis",
                    "Trichinella"
                   "Norovirus",
                   "Rotavirus",
                   "Astrovirus",
                   "Hepatitis A"]
    result = ""
    for item in bacteria_lst:
        if item.lower() in text.lower():
            result = item
            break
    if result == "":
        result = None
    return result

def replacing_food_category(text):
    lower_text = str(text).lower()
    if "risk" in lower_text:
        text = "Risk"
    elif "surveillance" in lower_text:
        text = "Surveillance"
    else:
        text = None
    return text

In [11]:
def get_unit(text):
    
    try:
        match_per_units = re.findall('\w+/\w+', text)
        match_single_units = re.findall('\w+$', text)
        if match_per_units:
            result = match_per_units[0]
            result = re.sub("\d+", "", result)
        elif match_single_units:
            result = match_single_units[0]
            result = re.sub("\d+", "", result)
        else:
            result = "none"
        return result.lower()
    except Exception as e:
        return "none"

In [12]:
def get_decimal(text):
    decimal_pattern = "\d+\.\d+"
    result = re.search(decimal_pattern, text)
    return result.group()

def get_digit(text):
    digit_pattern = "\d+"
    result= re.search(digit_pattern, text)
    result =  result.group()    
    return result

def get_digit_with_unit(text):
    digit_with_unit_pattern = "\d+\w+"
    result = re.search(digit_with_unit_pattern, text)
    res_sub = re.sub("[g|kg|cfu|mg]","",result.group())
    return res_sub

def get_digit_with_pre_pattern(text):
    digit_with_pre_pattern = "\/\d+\w+"
    result = re.search(digit_with_pre_pattern, text)
    result = result.group()
    res_sub = re.sub("[/|g|<|>]", "", result)
    return res_sub

def get_result_num(text):
    try:
        digit_pattern = re.compile("\d+")
        decimal_pattern = re.compile("\d+\.\d+")
        digit_with_unit_pattern = re.compile("\d+\w+")
        digit_with_pre_pattern = re.compile("\/\d+\w+")
        find_match = re.findall("\d+\s+\w+\/\w+", text)
        if decimal_pattern.match(text):
            result = get_decimal(text)
        elif find_match:
            result = get_digit(find_match[0])
        elif digit_with_pre_pattern.match(text):
            result =get_digit_with_pre_pattern(text)
        elif digit_with_unit_pattern.match(text):
            result = get_digit_with_unit(text)
        elif digit_pattern.match(text):
            result = get_digit(text)
        else:
            pattern = "\d+"
            result = re.findall(pattern, text)
            result = result[-1]
        result = pd.to_numeric(result, errors = 'coerce')
        return result
    except Exception as e:
        return "None"

In [13]:
def test_split(text):
    if "–" in str(text):
        res = re.split("–", str(text))
        res = res[0].strip()
    elif "-" in str(text):
        res = re.split("-", str(text))
        res = res[0].strip()
    else:
        res = text
    first_part = res
    return first_part

def reason_split(text):
    if "–" in str(text):
        res = re.split("–", str(text))
        res = res[1].strip()
    elif "-" in str(text):
        res = re.split("-", str(text))
        res = res[1].strip()
    else:
        res = text
    second_part = res
    return second_part

## Alternative of renaming the columns
Just assign it to the .columns attribute:

df = pd.DataFrame({'$a':[1,2], '$b': [10,20]})

df
   $a  $b
   
0   1  10

1   2  20

df.columns = ['a', 'b']

df

   a   b
   
0  1  10

1  2  20


link: https://stackoverflow.com/questions/11346283/renaming-column-names-in-pandas

In [14]:
from string import punctuation
def rename_column__by_replacing_white_space(df):
    rename_column_map = {}

    for column in df.columns:
        try:
            rename_column = column.lower().replace("-", " ").replace(punctuation, "").replace(" ", "_").replace("", "")
            rename_column_map[column] = rename_column
        except Exception as e:
            df.drop(column, inplace = True)
    
    return rename_column_map

def rename_column__by_replacing_white_space_and_drop_none(df):
    new_column_lst = []
    for index, column in enumerate(list(df.columns)):
        column_str = str(column)
        rename_column = column_str.lower().strip().replace("-", " ").replace(punctuation, "").replace(" ", "_").replace("", "")
        if "risk" in rename_column or "surveillance" in rename_column:
            rename_column = "food_category"
        rename_column = re.sub("[|ï|»|¿|:]", "", rename_column)
        new_column_lst.append(str(rename_column))
    return new_column_lst

In [15]:
def get_info(data):
    """
    Get the information about the data such as columns, info, and shape in one go
    """
    print("Columns:")
    print("-------------------")
    print(data.columns)
    print("\nInfo")
    print("-------------------")
    print(data.info())
    print("\nShape")
    print(data.shape)

def get_columns_unique(df_yearly):
    for col in list(df_yearly.columns):
        print(col)
        print(df_yearly[col].unique())
        print()
        
def view_data_info(df_yearly):
    get_info(df_yearly)
    get_columns_unique(df_yearly)

In [16]:
def test_split(text):
    if "–" in str(text):
        res = re.split("–", str(text))
        res = res[0].strip()
    elif "-" in str(text):
        res = re.split("-", str(text))
        res = res[0].strip()
    else:
        res = text
    first_part = res
    return first_part

In [17]:
def get_100_pct_null_rows(data):
    '''
    data: dataframe
    This function is to get the rows where it contains 100 percent of null values.
    It returns the index.
    '''
    null_rows_ind = get_null_rows(data)
    null_rows_df = data.loc[null_rows_ind]
    null100pct_rows_ind = []
    for index, row in null_rows_df.iterrows():
        if null_rows_df.loc[index].isnull().sum().sum() == null_rows_df.shape[1]:
            null100pct_rows_ind.append(index)
    return null100pct_rows_ind     

# 2023 Data

In [18]:
def read_multiple_csv(path_lst):
    df_yearly_list = [pd.read_csv(path) for path in path_lst]
    return df_yearly_list

def data_formatting(df_yearly):


    df_yearly.rename(columns = rename_column__by_replacing_white_space(df_yearly), inplace = True)

    df_yearly['date_of_fail'] = df_yearly['date_of_fail'].apply(convert_to_datetime)
    df_yearly['date_of_fail'] = df_yearly['date_of_fail'].fillna(method = 'ffill')

    df_yearly['result'] = df_yearly['results_/_units'].apply(get_result_num)

    df_yearly['result'] = pd.to_numeric(df_yearly['result'], errors = 'coerce')

    df_yearly['units'] = df_yearly['results_/_units'].apply(get_unit)
    df_yearly['units'] = df_yearly['units'].apply(lambda x:x.lower())
    
    df_yearly['results_2'] = df_yearly['units'].apply(get_result_num)

    df_yearly['test_applied'] = df_yearly['test_applied_/_reason_for_fail'].apply(test_split)
    
    df_yearly['reason_for_fail'] = df_yearly['test_applied_/_reason_for_fail'].apply(reason_split)

    df_yearly.drop(columns = ['test_applied_/_reason_for_fail', 'results_/_units', 'results_2'], inplace = True)
    
    df_yearly['food_category'] = df_yearly['food_category'].apply(replacing_food_category)
    
    df_yearly.loc[df_yearly['country_of_origin'].str.contains("Korea", na = False), 'country_of_origin'] = "Republic of Korea"
    df_yearly.loc[df_yearly['country_of_origin'].str.contains("Iran", na = False), 'country_of_origin'] = "Islamic Replubic of Iran"
    df_yearly.loc[df_yearly['country_of_origin'].str.contains("Tanzania", na = False), 'country_of_origin'] = "United Republic of Tanzania"
    
    
    df_yearly.loc[df_yearly['reason_for_fail'].str.contains("ipronil", na = False), 'reason_for_fail'] = "Fipronil"
    df_yearly.loc[df_yearly['reason_for_fail'].str.contains("isteria", na = False), 'reason_for_fail'] = 'Listeria'
    df_yearly.loc[df_yearly['reason_for_fail'].str.contains("orovirus", na = False), 'reason_for_fail'] = 'Norovirus'
    df_yearly.loc[df_yearly['reason_for_fail'].str.contains("lostridium perfringens", na = False), 'reason_for_fail'] = "Clostridium perfringens"
    df_yearly.loc[df_yearly['reason_for_fail'].str.contains("ampylobacter", na = False), 'reason_for_fail'] = "Campylobacter"
    
    df_yearly = df_yearly
    return df_yearly

def data_cleaning(df_yearly_path_list):    
    df_yearly = pd.concat(
        read_multiple_csv(df_yearly_path_list), 
        axis = 0)
    
    df_yearly = data_formatting(df_yearly)
    
    return df_yearly


In [19]:
df_2023_path_list = ["2023/01-imported-food-inspection-scheme-january-2023.csv",
                    "2023/02-imported-food-inspection-scheme-february-2023.csv",
                    "2023/03-imported-food-inspection-scheme-march-2023.csv",
                    "2023/04-imported-food-inspection-scheme-april-2023.csv"]
df_2023 = data_cleaning(df_2023_path_list)
df_2023.head()

,date_of_fail,product_description,country_of_origin,producer_name,reference,food_category,result,units,test_applied,reason_for_fail
0,2022-12-30,Soya bean curd,Vietnam,T&Z Trading Service Production Company,Imported Food Control Act Section 3(2),Surveillance,1000.0,cfu/g,Bacillus cereus detected in excess of levels p...,Bacillus cereus detected in excess of levels p...
1,2022-12-30,Soya bean curd,Vietnam,T&Z Trading Service Production Company,Imported Food Control Act Section 3(2),Surveillance,13000.0,cfu/g,Bacillus cereus detected in excess of levels p...,Bacillus cereus detected in excess of levels p...
2,2023-01-03,Giant taro (Alocasia macrorrhiza),Tonga,Salome Lousiale,FSC 1.4.4,Risk,NaN,none,Composition,Prohibited plant
3,2023-01-03,Chicken nuggets made from plants,United States of America,Impossible Foods Inc,FSC 1.3.2,Surveillance,NaN,none,Composition,Calcium Pantothenate (Vitamin B5) not permitte...
4,2023-01-05,Smoked dried fish mgebuka,United Republic of Tanzania,Banyenzeko Fish Export,Imported Food Control Act Section 3(2),Risk,NaN,none,The food is not suitable for its reasonable in...,goods infested with flies/maggots


In [20]:
view_data_info(df_2023)

Columns:
-------------------
Index(['date_of_fail', 'product_description', 'country_of_origin',
       'producer_name', 'reference', 'food_category', 'result', 'units',
       'test_applied', 'reason_for_fail'],
      dtype='object')

Info
-------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 101 entries, 0 to 21
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date_of_fail         101 non-null    datetime64[ns]
 1   product_description  101 non-null    object        
 2   country_of_origin    101 non-null    object        
 3   producer_name        101 non-null    object        
 4   reference            101 non-null    object        
 5   food_category        101 non-null    object        
 6   result               78 non-null     float64       
 7   units                101 non-null    object        
 8   test_applied         101 non-null    object        
 9   reas

# 2022 Data

In [21]:
from pandasql import sqldf

pysqldf = lambda q:sqldf(q)

In [22]:
df_2022_path_list = ['2022/01-imported-food-inspection-scheme-january-2022.csv',
                    '2022/02-imported-food-inspection-scheme-february-2022.csv',
                    '2022/03-imported-food-inspection-scheme-march-2022.csv',
                    '2022/04-imported-food-inspection-scheme-april-2022.csv',
                    '2022/05-imported-food-inspection-scheme-may-2022.csv',
                    '2022/06-imported-food-inspection-scheme-june-2022.csv',
                    '2022/07-imported-food-inspection-scheme-july-2022.csv',
                    '2022/08-imported-food-inspection-scheme-august-2022.csv',
                    '2022/09-imported-food-inspection-scheme-september-2022.csv',
                    '2022/10-imported-food-inspection-scheme-october-2022.csv',
                    '2022/11-imported-food-inspection-scheme-november-2022.csv',
                    '2022/12-imported-food-inspection-scheme-december-2022.csv']

df_2022 = data_cleaning(df_2022_path_list)
df_2022.head()

,category,date_of_fail,product_description,country_of_origin,producer_name,test_applied,reason_for_fail,result/s,units,reference,holding_order_number,category,test_applied/_reason_for_fail,results/units,food_category,results_/_units.1,result
0,Risk,2022-01-04,Sesame seed,India,Pramukh Food Exports Pvt Ltd,NaN,NaN,Detected,none,SES 05/2014,NaN,NaN,NaN,NaN,None,NaN,NaN
1,Surveillance,2022-01-05,Cream crackers,Malaysia,Mondelez Malaysia Sales Snd Bhd,NaN,NaN,NaN,none,FSC 1.3.2,HO match 17378,NaN,NaN,NaN,None,NaN,NaN
2,Risk,2022-01-06,Dried herring,Philippines,Yang Yang Food Products,NaN,NaN,513,none,HIS 12/2016,NaN,NaN,NaN,NaN,None,NaN,NaN
3,Risk,2022-01-07,Dried seaweed,China,Tesana Trading Ltd (Hong Kong),NaN,NaN,3440,none,BSW 12/2016,NaN,NaN,NaN,NaN,None,NaN,NaN
4,Risk,2022-01-10,Roasted sesame seed,Nepal,Sagoon Export Nepal Pvt Ltd,NaN,NaN,NaN,none,Imported Food Control Act Section 3(2),NaN,NaN,NaN,NaN,None,NaN,NaN


In [23]:
view_data_info(df_2022)

Columns:
-------------------
Index(['category   ', 'date_of_fail', 'product_description',
       'country_of_origin', 'producer_name', 'test_applied', 'reason_for_fail',
       'result/s', 'units', 'reference', 'holding_order_number', 'category',
       'test_applied/_reason_for_fail', 'results/units', 'food_category',
       'results_/_units.1', 'result'],
      dtype='object')

Info
-------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 296 entries, 0 to 20
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   category                       23 non-null     object        
 1   date_of_fail                   296 non-null    datetime64[ns]
 2   product_description            296 non-null    object        
 3   country_of_origin              296 non-null    object        
 4   producer_name                  296 non-null    object        
 5   test_applied  

In [24]:
df_2022.rename(columns = {"food_category_full": "food_category"}, inplace = True)
get_info(df_2022)

Columns:
-------------------
Index(['category   ', 'date_of_fail', 'product_description',
       'country_of_origin', 'producer_name', 'test_applied', 'reason_for_fail',
       'result/s', 'units', 'reference', 'holding_order_number', 'category',
       'test_applied/_reason_for_fail', 'results/units', 'food_category',
       'results_/_units.1', 'result'],
      dtype='object')

Info
-------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 296 entries, 0 to 20
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   category                       23 non-null     object        
 1   date_of_fail                   296 non-null    datetime64[ns]
 2   product_description            296 non-null    object        
 3   country_of_origin              296 non-null    object        
 4   producer_name                  296 non-null    object        
 5   test_applied  

# 2021 Data

In [25]:
def getting_new_column(df_monthly):    
    new_cols = []
    for col in list(df_monthly.columns):
        lower_col = col.lower()
        if "risk" in lower_col:
            lower_col = "risk"
        new_col = lower_col.replace(":", "").replace(" ", "_")
        new_cols.append(new_col)
    return new_cols

In [26]:
def get_section(df_monthly):
    df_monthly_null_row_ind = get_null_rows(df_monthly)
    section_row = []
    for index , row in df_monthly.iterrows():
        try:
            if index in df_monthly_null_row_ind and str(list(row)[0]) != "nan":
                section_row.append((index, list(row)[0]))

        except Exception as e:
            if index == df_monthly_null_row_ind and str(list(row)[0]) != "nan":
                section_row.append((index, list(row)[0]))
    return section_row

def get_food_category(df_monthly):
    section_row = get_section(df_monthly)
    food_category_lst = []
    for i in range(df_monthly.shape[0]):
        for tup in section_row:
            if i >= tup[0]:
                food_category_lst.append(tup[1])
                break
            else:
                food_category_lst.append(df_monthly.columns[0])
                break
    return food_category_lst



In [27]:
def formatting_data_monthly(df_monthly):
    df_monthly['food_category'] =get_food_category(df_monthly)
    df_monthly.columns = getting_new_column(df_monthly)
    return df_monthly

In [28]:
def get_null_rows(data):
    '''
    data: DataFrame
    this function is to get the row where
    '''
    drop_index = []
    for i, r in data.iterrows():
        if (data.loc[i].isnull().sum().sum() == data.shape[1]
        or data.loc[i].isnull().sum().sum() >= int(data.shape[1]*0.70)):
            drop_index.append(i)
    if len(drop_index) == 1:
        drop_index = drop_index[0]
        
    return drop_index

def get_row_column(df_yearly):
    '''
    df_yearly: DataFrame Yearly
    Get the row where it contain column.
    It returns the index of the row
    '''
    column_ind_lst = []
    for index, row in df_yearly.iterrows():
        row_list = list(row)
        first_list = str(row_list[0])
        row_list_lower = first_list.lower()
        if "date of fail" in row_list_lower:
            column_ind_lst.append(index)
    return column_ind_lst

In [29]:
def concat_untidy_data(df_yearly_path_list):
    df_yearly_list = []
    for path in df_yearly_path_list:
        df = pd.read_csv(path, encoding = "latin-1")
        df = formatting_data_monthly(df)
        df_yearly_list.append(df)

    df_yearly = pd.concat(df_yearly_list, axis = 0, join = 'outer')
    df_yearly = df_yearly.reset_index()
    df_yearly = df_yearly.drop(columns = "index")
    return df_yearly

In [30]:
def get_dfs_to_concatenate(df_yearly):
    '''
    df_yearly: DataFrame Yearly
    Get the dataframes to be concatenated
    It returns the list of dfs 
    '''
    df_yearly_lst = []
    column_df = df_yearly.loc[get_row_column(df_yearly)]
    column_ind_lst = column_df.index

    for index, col_ind in enumerate(column_ind_lst):
        if index != len(column_ind_lst)-1:

            df = df_yearly.loc[(column_ind_lst[index]+1):(column_ind_lst[index+1]-2)]        
        else :

            df = df_yearly.loc[(column_ind_lst[index]+1):]
        current_columns = list(df.columns)
        new_columns = list(df_yearly.loc[column_ind_lst[index]])
        columns_mapper_dict = {}
        for i in list(range(0,len(current_columns))):
            columns_mapper_dict[current_columns[i]] = new_columns[i]

        df = df.rename(columns = columns_mapper_dict)
        new_columns = rename_column__by_replacing_white_space_and_drop_none(df)
        df.columns = new_columns
        if 'nan' in list(df.columns) :
            df.drop(columns = 'nan', inplace = True)
        df_yearly_lst.append(df)
    return df_yearly_lst

In [31]:
def data_formatting_v2(df_yearly):
    df_yearly['date_of_fail'] = df_yearly['date_of_fail'].apply(convert_to_datetime)
    df_yearly['date_of_fail'] = df_yearly['date_of_fail'].fillna(method = 'ffill')

    df_yearly.rename(columns = {'result/s':'results'}, inplace = True)
    df_yearly['results'] = pd.to_numeric(df_yearly['results'], errors = 'coerce')

    df_yearly['results_2'] = df_yearly['units'].apply(get_result_num)
    df_yearly['results'] = df_yearly['results'].fillna(df_yearly['results_2'])
    df_yearly['results'] = pd.to_numeric(df_yearly['results'], errors = 'coerce')
    
    df_yearly['food_category'] = df_yearly['food_category'].apply(replacing_food_category)

    df_yearly['units'] = df_yearly['units'].apply(get_unit)
    df_yearly['units'] = df_yearly['units'].apply(lambda x:x.lower())
    
    
    df_yearly.loc[df_yearly['country_of_origin'].str.contains("Korea", na = False), 'country_of_origin'] = "Republic of Korea"
    df_yearly.loc[df_yearly['country_of_origin'].str.contains("Iran", na = False), 'country_of_origin'] = "Islamic Replubic of Iran"
    df_yearly.loc[df_yearly['country_of_origin'].str.contains("Tanzania", na = False), 'country_of_origin'] = "United Republic of Tanzania"
    
    df_yearly.loc[df_yearly['reason_for_fail'].str.contains("ipronil", na = False), 'reason_for_fail'] = "Fipronil"
    df_yearly.loc[df_yearly['reason_for_fail'].str.contains("isteria", na = False), 'reason_for_fail'] = 'Listeria'
    df_yearly.loc[df_yearly['reason_for_fail'].str.contains("orovirus", na = False), 'reason_for_fail'] = 'Norovirus'
    df_yearly.loc[df_yearly['reason_for_fail'].str.contains("lostridium perfringens", na = False), 'reason_for_fail'] = "Clostridium perfringens"
    df_yearly.loc[df_yearly['reason_for_fail'].str.contains("ampylobacter", na = False), 'reason_for_fail'] = "Campylobacter"
    
    
    df_yearly = df_yearly.drop(columns = "results_2")
    return df_yearly

In [32]:
def data_cleaning_v2(df_yearly_path_list):    
    df_yearly_untidy = concat_untidy_data(df_yearly_path_list)

    df_yearly_lst = get_dfs_to_concatenate(df_yearly_untidy)

    df_yearly = pd.concat(df_yearly_lst, axis = 0, join = 'outer')

    df_yearly = data_formatting_v2(df_yearly)
    
    return df_yearly

In [33]:
df_2021_path_list = ['2021/01 january 2021.csv',
                    '2021/02 february 2021.csv',
                    '2021/03 march 2023.csv',
                    '2021/04 april 2021.csv',
                    '2021/05 may 2021.csv',
                    '2021/06 june 2021.csv',
                    '2021/07 july 2021.csv',
                    '2021/08 august 2021.csv',
                    '2021/09 september 2021.csv',
                    '2021/10 october 2021.csv',
                    '2021/11 november 2021.csv',
                    '2021/12 december 2021.csv']
df_2021 = data_cleaning_v2(df_2021_path_list)
df_2021

,date_of_fail,product_description,country_of_origin,producer_name,reason_for_fail,results,units,reference,food_category,test_applied,holding_order_number
1,2020-12-29,Areca catechu (Betel nut),India,Unique Fragrances,Prohibited plant,NaN,none,FSC 1.4.4,Risk,NaN,NaN
2,2021-01-05,Areca catechu (Betel nut),India,Kirti Impex,Prohibited plant,NaN,none,FSC 1.4.4,Risk,NaN,NaN
3,2021-01-06,Chilli powder,India,Meximport Food Win Se Rl,Salmonella,125.00,g,SPI 06/2019,Risk,NaN,NaN
4,2021-01-07,Dried kelp,Republic of Korea,Badamoolsan Fishing Union Corporation,Iodine,2660.00,mg/kg,BSW 12/2016,Risk,NaN,NaN
5,2021-01-08,Dried seaweed,Republic of Korea,Badamoolsan Fishing Union Corporation,Iodine,4260.00,mg/kg,BSW 12/2016,Risk,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
469,2021-12-23,Black harmula,Islamic Replubic of Iran,Barg Sabz Co,Prohibited plant â Peganum harmala,NaN,none,FSC 1.4.4,Surveillance,Composition,17555
470,2021-12-29,Frozen dry shrimp dumpling,China,Yunnan Moben International Trade Co Ltd,Prohibited plant - Amanita muscaria (Agaric)Â,NaN,none,FSC 1.4.4,Surveillance,Composition,17554
471,2021-12-30,Dried dates,China,Hong Fung Trading Co (Hong Kong),Cypermethrin detected in excess of MRL,0.06,mg/kg,FSC 1.4.2,Surveillance,Fruit and vegetable residue screen,17551
472,2021-12-30,Dried dates,China,Hong Fung Trading Co (Hong Kong),Fenvalerate detected in excess of MRL,0.08,mg/kg,NaN,Surveillance,Fruit and vegetable residue screen,NaN


In [34]:
view_data_info(df_2021)

Columns:
-------------------
Index(['date_of_fail', 'product_description', 'country_of_origin',
       'producer_name', 'reason_for_fail', 'results', 'units', 'reference',
       'food_category', 'test_applied', 'holding_order_number'],
      dtype='object')

Info
-------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 427 entries, 1 to 473
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date_of_fail          427 non-null    datetime64[ns]
 1   product_description   402 non-null    object        
 2   country_of_origin     402 non-null    object        
 3   producer_name         402 non-null    object        
 4   reason_for_fail       414 non-null    object        
 5   results               179 non-null    float64       
 6   units                 427 non-null    object        
 7   reference             408 non-null    object        
 8   food_category         427

# 2020 Food Import Data

##### Gather the file path

In [35]:
data_2020_path_lst = ["2020/01-january-2020.csv","2020/02-february-2020.csv", "2020/03-march-2020.csv", "2020/04-april-2020.csv"
             ,"2020/05-may-2020.csv", "2020/06-june-2020.csv", "2020/07-july-2020.csv", "2020/08-august-2020.csv",
            "2020/09-september-2020.csv", "2020/10-october-2020.csv", "2020/11-november-2020.csv", "2020/12-december-2020.csv"
            ]

df_2020 = data_cleaning_v2(data_2020_path_lst)
df_2020.head()

,date_of_fail,product_description,country_of_origin,producer_name,reason_for_fail,results,units,reference,food_category,test_applied,holding_order_number
1,2020-01-07,Frozen cooked black tiger prawns,Vietnam,Trong Nhan Seafood Company,Vibrio alginolyticus,125.0,g,"Section3, (2)(i) of the Imported Food Control ...",Risk,NaN,NaN
2,2020-01-09,Cooked pasted crab,Vietnam,Hai Chau Aquatic Product Company Limited,Standard plate count,NaN,cfu/g,CRU 03/2019,Risk,NaN,NaN
3,2020-01-21,Black pepper,Islamic Replubic of Iran,Golha Food Industries,Salmonella,125.0,g,SPI 06/2019,Risk,NaN,NaN
4,2020-01-21,Gorgonzola dolce intero DOP cheese,Italy,La Casearia Carpenedo,Listeria,125.0,g,CHE 07/2018,Risk,NaN,NaN
5,2020-01-28,Crushed chilli,Myanmar,H-Htet Company Limited,Salmonella,125.0,g,SPI 06/2019,Risk,NaN,NaN


In [36]:
view_data_info(df_2020)

Columns:
-------------------
Index(['date_of_fail', 'product_description', 'country_of_origin',
       'producer_name', 'reason_for_fail', 'results', 'units', 'reference',
       'food_category', 'test_applied', 'holding_order_number'],
      dtype='object')

Info
-------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 539 entries, 1 to 585
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date_of_fail          539 non-null    datetime64[ns]
 1   product_description   504 non-null    object        
 2   country_of_origin     501 non-null    object        
 3   producer_name         501 non-null    object        
 4   reason_for_fail       503 non-null    object        
 5   results               237 non-null    float64       
 6   units                 539 non-null    object        
 7   reference             502 non-null    object        
 8   food_category         539

# Data 2019

In [37]:
df_2019_path_lst = ['2019/06 june 2019.csv', '2019/07 july 2019.csv', '2019/08 august 2019.csv',
                '2019/09 september 2019.csv', '2019/10 october 2019.csv', '2019/11 november 2019.csv',
                '2019/12 december 2019.csv']

df_2019 = data_cleaning_v2(df_2019_path_lst)
df_2019.head()

,date_of_fail,product_description,country_of_origin,producer_name,reason_for_fail,results,units,reference,food_category,test_applied,holding_order_number
1,2019-06-05,Kashmiri chilli powder,India,Anna Aluminium Co Pvt Ltd,Salmonella,125.0,g,SPI 06/2019,Risk,NaN,NaN
2,2019-06-11,Frozen cooked prawns,Vietnam,Camau Seafood Processing & Service Jointstock ...,Standard plate count,NaN,cfu/g,CRU 03/2019,Risk,NaN,NaN
3,2019-06-13,Chocolate halawa,Saudi Arabia,Qoot Food,Salmonella,125.0,g,SES 05/2014,Risk,NaN,NaN
4,2019-06-17,Frozen cooked vannamei prawns,Vietnam,Thalassa Seafoods (Vietnam),Standard plate count,NaN,cfu/g,CRU 03/2019,Risk,NaN,NaN
5,2019-06-14,Marinara mix,Malaysia,Labuan Food Industries Sdn Bhd,Salmonella,125.0,g,MSM 11/2018,Risk,NaN,NaN


In [38]:
view_data_info(df_2019)

Columns:
-------------------
Index(['date_of_fail', 'product_description', 'country_of_origin',
       'producer_name', 'reason_for_fail', 'results', 'units', 'reference',
       'food_category', 'test_applied', 'holding_order_number'],
      dtype='object')

Info
-------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 246 entries, 1 to 282
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date_of_fail          246 non-null    datetime64[ns]
 1   product_description   228 non-null    object        
 2   country_of_origin     228 non-null    object        
 3   producer_name         228 non-null    object        
 4   reason_for_fail       228 non-null    object        
 5   results               147 non-null    float64       
 6   units                 246 non-null    object        
 7   reference             222 non-null    object        
 8   food_category         246

# Combining all of the years

In [39]:
df_2022

,category,date_of_fail,product_description,country_of_origin,producer_name,test_applied,reason_for_fail,result/s,units,reference,holding_order_number,category,test_applied/_reason_for_fail,results/units,food_category,results_/_units.1,result
0,Risk,2022-01-04,Sesame seed,India,Pramukh Food Exports Pvt Ltd,NaN,NaN,Detected,none,SES 05/2014,NaN,NaN,NaN,NaN,None,NaN,NaN
1,Surveillance,2022-01-05,Cream crackers,Malaysia,Mondelez Malaysia Sales Snd Bhd,NaN,NaN,NaN,none,FSC 1.3.2,HO match 17378,NaN,NaN,NaN,None,NaN,NaN
2,Risk,2022-01-06,Dried herring,Philippines,Yang Yang Food Products,NaN,NaN,513,none,HIS 12/2016,NaN,NaN,NaN,NaN,None,NaN,NaN
3,Risk,2022-01-07,Dried seaweed,China,Tesana Trading Ltd (Hong Kong),NaN,NaN,3440,none,BSW 12/2016,NaN,NaN,NaN,NaN,None,NaN,NaN
4,Risk,2022-01-10,Roasted sesame seed,Nepal,Sagoon Export Nepal Pvt Ltd,NaN,NaN,NaN,none,Imported Food Control Act Section 3(2),NaN,NaN,NaN,NaN,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,NaN,2022-12-22,Oyster,China,Easy Well (China) Ltd,Escherichia coli detected in excess of levels ...,Escherichia coli detected in excess of levels ...,NaN,mpn/g,MOL 12/2021,NaN,NaN,NaN,NaN,Risk,NaN,110.0
17,NaN,2022-12-23,Chilli powder,India,Mahashian De Hatti Limited,Salmonella detected,Salmonella detected,NaN,g,SPI 06/2019,NaN,NaN,NaN,NaN,Risk,NaN,125.0
18,NaN,2022-12-29,Vermicelli,India,Bhatia Kakaar Enterprises,Composition,Vitamin B12 not permitted in this food,NaN,none,FSC 1.3.2,NaN,NaN,NaN,NaN,Surveillance,NaN,NaN
19,NaN,2022-12-29,Macaroni,India,Bhatia Kakaar Enterprises,Composition,Vitamin B12 not permitted in this food,NaN,none,FSC 1.3.2,NaN,NaN,NaN,NaN,Surveillance,NaN,NaN


In [40]:
df_combined_list = [df_2019, df_2020, df_2021, df_2022, df_2023]
for df in df_combined_list:
    print(df.shape[0])
df_combined = pd.concat(df_combined_list, axis = 0, join = 'outer')
df_combined.reset_index(inplace = True)
df_combined.drop(columns = 'index', inplace = True)
df_combined.head()

246
539
427
296
101


,date_of_fail,product_description,country_of_origin,producer_name,reason_for_fail,results,units,reference,food_category,test_applied,holding_order_number,category,result/s,category,test_applied/_reason_for_fail,results/units,results_/_units.1,result
0,2019-06-05,Kashmiri chilli powder,India,Anna Aluminium Co Pvt Ltd,Salmonella,125.0,g,SPI 06/2019,Risk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-06-11,Frozen cooked prawns,Vietnam,Camau Seafood Processing & Service Jointstock ...,Standard plate count,NaN,cfu/g,CRU 03/2019,Risk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-06-13,Chocolate halawa,Saudi Arabia,Qoot Food,Salmonella,125.0,g,SES 05/2014,Risk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-06-17,Frozen cooked vannamei prawns,Vietnam,Thalassa Seafoods (Vietnam),Standard plate count,NaN,cfu/g,CRU 03/2019,Risk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-06-14,Marinara mix,Malaysia,Labuan Food Industries Sdn Bhd,Salmonella,125.0,g,MSM 11/2018,Risk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
view_data_info(df_combined)

Columns:
-------------------
Index(['date_of_fail', 'product_description', 'country_of_origin',
       'producer_name', 'reason_for_fail', 'results', 'units', 'reference',
       'food_category', 'test_applied', 'holding_order_number', 'category   ',
       'result/s', 'category', 'test_applied/_reason_for_fail',
       'results/units', 'results_/_units.1', 'result'],
      dtype='object')

Info
-------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1609 entries, 0 to 1608
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   date_of_fail                   1609 non-null   datetime64[ns]
 1   product_description            1531 non-null   object        
 2   country_of_origin              1528 non-null   object        
 3   producer_name                  1528 non-null   object        
 4   reason_for_fail                1434 non-null   object        
 5   

In [42]:
df_combined['test_applied'] = df_combined['test_applied'].fillna("Not Applied")
df_combined['test_applied'].value_counts()

Not Applied                                                                                     684
Composition                                                                                     361
Fruit and vegetable residue screen                                                              254
Fruit & vegetable residue screen                                                                 51
Aflatoxin detected in excess of level permitted                                                  27
                                                                                               ... 
Cronobacter detected                                                                              1
Nitrofurans                                                                                       1
Aflatoxin detected in excess of level permitted (being 0.005 mg/kg as 35% peanut in product)      1
 Fruit and vegetable residue screen                                                               1


In [43]:
reason_for_fail_val_count = dict(df_combined['reason_for_fail'].value_counts())
least_frequent_drop_index = []
for k, v in reason_for_fail_val_count.items():
    if v >= 5:
        df_combined.loc[df_combined['reason_for_fail'].str.contains(k, na = False), "reason_for_fail"] = k

C:\Users\darry\AppData\Local\Temp\ipykernel_41428\3360836952.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_combined.loc[df_combined['reason_for_fail'].str.contains(k, na = False), "reason_for_fail"] = k


In [44]:

df_combined['results_in_mg_per_kg'] = df_combined.apply(
    lambda x:x['results']*10000 if x['units'] == "g" else(x['results'] * 100000000 if x['units'] == 'mpn/g' else(x['results'] if x['units'] == 'ppm' else(x['results'] ))),
    axis = 1
)
df_combined['results_in_mg_per_kg'].unique()

array([1.250e+06,       nan, 1.100e+01, 7.000e+02, 2.300e-02, 3.000e-02,
       1.900e-01, 7.000e-02, 5.500e-01, 5.300e-02, 1.200e+00, 2.000e-01,
       1.000e-01, 3.100e-01, 5.000e-02, 9.000e-02, 3.700e+02, 1.200e+01,
       4.400e-02, 7.600e-02, 2.900e+02, 2.000e+01, 1.800e+01, 2.800e-02,
       8.600e-02, 6.000e-02, 3.000e-01, 3.700e+01, 3.100e-03, 2.600e-02,
       2.100e-01, 8.700e-02, 7.300e-02, 1.600e-01, 2.140e-01, 3.600e+02,
       3.900e+03, 3.300e+03, 2.000e+03, 5.320e+02, 3.900e+02, 2.300e-01,
       1.100e+00, 1.400e-01, 1.200e-01, 3.400e+03, 2.800e+02, 2.500e-02,
       2.100e+02, 6.660e+03, 1.100e-01, 3.090e+02, 2.180e+02, 1.500e+03,
       4.730e+02, 5.640e+02, 2.530e+02, 2.760e+02, 3.080e+02, 4.800e+02,
       2.400e+01, 2.100e+01, 5.400e-02, 2.200e-02, 3.100e-02, 6.800e-02,
       2.700e+00, 3.000e+03, 4.000e+03, 3.300e-02, 1.700e-02, 2.700e-02,
       3.500e+03, 2.800e-03, 9.900e-01, 4.800e-02, 1.459e+03, 7.100e+02,
       4.390e+02, 2.340e+02, 4.850e+02, 2.650e+02, 

In [45]:
df_combined.loc[df_combined['test_applied'].str.contains("permitted"), 'test_applied'] = "Not Applied"
df_combined.loc[df_combined['test_applied'].str.contains("detected"), 'test_applied'] = "Not Applied"

df_combined.loc[df_combined['reason_for_fail'].str.contains("ipronil", na = False), 'reason_for_fail'] = "Fipronil"
df_combined.loc[df_combined['reason_for_fail'].str.contains("isteria", na = False), 'reason_for_fail'] = 'Listeria'
df_combined.loc[df_combined['reason_for_fail'].str.contains("orovirus", na = False), 'reason_for_fail'] = 'Norovirus'
df_combined.loc[df_combined['reason_for_fail'].str.contains("lostridium perfringens", na = False), 'reason_for_fail'] = "Clostridium perfringens"
df_combined.loc[df_combined['reason_for_fail'].str.contains("ampylobacter", na = False), 'reason_for_fail'] = "Campylobacter"

In [46]:
df_combined.to_csv("food_imported_data.csv")
df_combined.to_csv("food_imported_data_view.csv")